In [1]:
import time

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import errno

from common.camera import *
from common.model import *
from common.loss import *
from common.utils import *
from common.visualization import *
from common.generators import ChunkedGenerator, UnchunkedGenerator
from common.utils import deterministic_random

from scripts.data_preprocessing_human36 import *
from scripts.build_model import *
from scripts.train import *
from scripts.eval import *

## Load and Preprocess Data

In [2]:
dataset_path = "../data/data_3d_h36m.npz"
dataset_path_2d = '../data/data_2d_h36m_cpn_ft_h36m_dbb.npz'

In [3]:
dataset, keypoints = load_and_preprocess_human36m(dataset_path, dataset_path_2d)

load and process 3d data
load and process 2d data


## Generate Subsets

In [4]:
subjects_test = ['S5']

In [5]:
cameras_valid, poses_valid, poses_valid_2d = fetch(dataset = dataset, 
                                                   keypoints = keypoints, 
                                                   subjects = subjects_test, 
                                                   stride = 1)

## Load Model

In [6]:
resume = True
filter_widths = [3,3,3,3,3]
model_pos_train, model_pos, model_traj, model_traj_train = build_models(17, 2, 17, filter_widths)

# chk_filename = "../checkpoint/pretrained_h36m_cpn.bin"
chk_filename = "checkpoints/epoch_80.bin"
print('Loading checkpoint', chk_filename)
checkpoint = torch.load(chk_filename, map_location=lambda storage, loc: storage)
print('This model was trained for {} epochs'.format(checkpoint['epoch']))
model_pos_train.load_state_dict(checkpoint['model_pos'])
model_pos.load_state_dict(checkpoint['model_pos'])
model_traj_train.load_state_dict(checkpoint['model_traj'])
model_traj.load_state_dict(checkpoint['model_traj'])

Loading checkpoint checkpoints/epoch_80.bin
This model was trained for 80 epochs


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Test Data Generator

In [7]:
receptive_field = model_pos.receptive_field()
print('INFO: Receptive field: {} frames'.format(receptive_field))
pad = (receptive_field - 1) // 2 # Padding on each side

INFO: Receptive field: 243 frames


test data

In [8]:
test_generator = UnchunkedGenerator(cameras_valid[:1], 
                                    poses_valid[:1], 
                                    poses_valid_2d[:1], 
                                    pad=pad, 
                                    causal_shift=0, 
                                    augment=False)

In [9]:
# input_keypoints = keypoints["S9"]["Walking"][0].copy()
# gt = dataset["S9"]["Walking"]["positions_3d"][0].copy()
# cam = dataset["S9"]["Walking"]["cameras"][0].copy()

# test_generator = UnchunkedGenerator([cam], 
#                                     None, 
#                                     [input_keypoints],
#                                     pad=pad, 
#                                     causal_shift=0, 
#                                     augment=False)

## Inference

In [10]:
with torch.no_grad():
    model_pos.eval()
    N = 0
    for cam_, batch, batch_2d in test_generator.next_epoch():
        inputs_2d = torch.from_numpy(batch_2d.astype('float32'))
        inputs_3d = torch.from_numpy(batch.astype('float32'))
        cam_ = torch.from_numpy(cam_.astype('float32'))
        # Positional model
        predicted_3d_pos = model_pos(inputs_2d)    
        prediction = predicted_3d_pos.numpy()
        gt = inputs_3d.numpy()
        gt_2d = inputs_2d.numpy()

In [11]:
prediction = prediction[0]
gt = gt[0]
gt_2d = gt_2d[0]

## Post Processing

In [12]:
trajectory = gt[:, :1]
gt[:, 1:] += trajectory
prediction += trajectory

get camera infos

In [13]:
cam = dataset.cameras()["S5"][0]

In [14]:
# convert from camera coordinates to world coordinates
prediction = camera_to_world(prediction, R=cam['orientation'], t=cam['translation'])
gt = camera_to_world(gt, R=cam['orientation'], t=cam['translation'])

## Visualization

In [15]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [16]:
human36m_kpts_name = ['Pelvis', 'RHip', 'RKnee', 'RAnkle','LHip','LKnee',
                      'LAnkle','Spine1','Neck', 'Head','Site','LShoulder',
                      'LElbow','LWrist','RShoulder', 'RElbow','RWrist']

In [17]:
# ground truth 3d
frames, layout = generate_frames_layout(gt[::100], dataset.skeleton(), human36m_kpts_name)
fig = dict(data=get_line_3d_segs(gt, 0, dataset.skeleton(), human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [18]:
# projection 2d
gt_proj_2d = project_to_2d(inputs_3d, cam_)
frames, layout = generate_frames_layout_2d(gt_proj_2d[0][::100], dataset.skeleton(), human36m_kpts_name, x_range=[-1, 1], y_range=[1, -1])
fig = dict(data=get_line_2d_segs(gt_proj_2d[0], 0, dataset.skeleton(), human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [19]:
# prediction 3d
frames, layout = generate_frames_layout(prediction[::100], dataset.skeleton(), human36m_kpts_name)
fig = dict(data=get_line_3d_segs(prediction, 0, dataset.skeleton(), human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)

In [20]:
# projection 2d
pred_proj_2d = project_to_2d(predicted_3d_pos, cam_)
frames, layout = generate_frames_layout_2d(pred_proj_2d[0][::100], dataset.skeleton(), human36m_kpts_name, x_range=[-1, 1], y_range=[1, -1])
fig = dict(data=get_line_2d_segs(pred_proj_2d[0], 0, dataset.skeleton(), human36m_kpts_name), layout=layout, frames=frames)
plotly.offline.iplot(fig)